In [1]:
from datetime import datetime
import pytz

def fecha_hora_lima():
    lima_tz = pytz.timezone('America/Lima')
    lima_time = datetime.now(lima_tz)
    return lima_time.strftime('%Y%m%d-%H%M%S')

def fecha_hora_lima_db():
    lima_tz = pytz.timezone('America/Lima')
    lima_time = datetime.now(lima_tz)
    return lima_time.strftime('%Y-%m-%d %H:%M:%S')

fecha_hora_para_archivos = fecha_hora_lima()
fecha_hora_para_base_de_datos = fecha_hora_lima_db()

In [2]:
import pandas as pd
import requests
import json

# URL del JSON
url = 'https://www.wong.pe/api/catalog_system/pub/category/tree/3'

# Solicitar el contenido JSON desde la URL
response = requests.get(url)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    data = response.json()  # Convertir la respuesta en un objeto JSON
    file_name = f'categories_{fecha_hora_para_archivos}.json'
    
    # Guardar el JSON en un archivo
    with open(file_name, 'w') as file:
        json.dump(data, file, indent=4)

    print('El archivo JSON se ha guardado exitosamente.')
    with open(file_name, 'r') as file:
        data = json.load(file)
    print(len(data)) 
else:
    print(f'Error al obtener el JSON. Código de estado: {response.status_code}')  

El archivo JSON se ha guardado exitosamente.
26


In [3]:
# file_path = 'wong_categories.json'
# 
# with open(file_path, 'r') as file:
#     data = json.load(file)
#     print(json.dumps(data, indent=4))

In [4]:
columns = ['id', 'name', 'hasChildren', 'url', 'NumChildren', 'Title', 'MetaTagDescription', 'parent_id']
df = pd.DataFrame(columns=columns)
df

,id,name,hasChildren,url,NumChildren,Title,MetaTagDescription,parent_id


In [5]:
for item in data:
  if item['hasChildren']:
    item['parent_id'] = ''
    item['NumChildren'] = len(item['children'])

    df.loc[len(df)] = item

    parent_id = item['id']

    for subitem in item['children']:
      subitem['parent_id'] = parent_id
      sub_parent_id = subitem['id']
      subitem['NumChildren'] = len(subitem['children'])

      df.loc[len(df)] = subitem

      if len(subitem['children']) > 0:
        for subsubitem in subitem['children']:
          subsubitem['parent_id'] = sub_parent_id
          subsubitem['NumChildren'] = len(subsubitem['children'])
          df.loc[len(df)] = subsubitem

In [6]:
df.query("id == 952")

,id,name,hasChildren,url,NumChildren,Title,MetaTagDescription,parent_id


In [7]:
file_name_categories_csv = f'wong_categories_{fecha_hora_para_archivos}.csv'
df.to_csv(file_name_categories_csv, index=False)

In [8]:
cats = df.query("parent_id == ''")['id'].to_list()
print(cats)

[800, 1200, 1400, 1700, 1800, 1900, 2000, 2100, 2200, 1000144, 1000003, 1000131, 1000143, 1000069, 1001074, 1001253, 1001327, 2301, 1001374, 1001402, 1001436, 1001467, 1001470, 1001673, 1001701]


In [9]:
import time

products = []

for cat in cats:
    start = 0
    end = 49
    
    while True:
        time.sleep(1)
        url = f"https://www.wong.pe/api/catalog_system/pub/products/search?fq=C:{cat}&_from={start}&_to={end}&O=OrderByPriceDESC"
        
        response = requests.get(url)
        
        if response.status_code == 200 or response.status_code == 206:
            data = response.json()
            if not data:
                break
                 
            products.extend(data)
            start += 50
            end += 50
        else:
            break
    
    print(f"status_code = {response.status_code}")
    print(f"cat = {cat}, start = {start}, end = {end}")

 
file_name_productos_csv = f'productos_{fecha_hora_para_archivos}.json'

with open(file_name_productos_csv, 'w') as file:
    json.dump(products, file, indent=4)

status_code = 200
cat = 800, start = 600, end = 649
status_code = 200
cat = 1200, start = 550, end = 599
status_code = 200
cat = 1400, start = 550, end = 599
status_code = 400
cat = 1700, start = 2550, end = 2599
status_code = 200
cat = 1800, start = 650, end = 699
status_code = 200
cat = 1900, start = 1250, end = 1299
status_code = 200
cat = 2000, start = 1000, end = 1049
status_code = 400
cat = 2100, start = 2550, end = 2599
status_code = 400
cat = 2200, start = 2550, end = 2599
status_code = 200
cat = 1000144, start = 300, end = 349
status_code = 200
cat = 1000003, start = 400, end = 449
status_code = 400
cat = 1000131, start = 2550, end = 2599
status_code = 200
cat = 1000143, start = 1250, end = 1299
status_code = 200
cat = 1000069, start = 650, end = 699
status_code = 200
cat = 1001074, start = 2150, end = 2199
status_code = 200
cat = 1001253, start = 1550, end = 1599
status_code = 200
cat = 1001327, start = 700, end = 749
status_code = 200
cat = 2301, start = 2300, end = 2349
sta

In [10]:
print(len(products))

28628


In [11]:
columns = ['productId', 'productName', 'brand', 'brandId', 'brandImageUrl','categoryId',
           'productTitle', 'metaTagDescription', 'categories', 'categoriesIds', 'link', 'Vendido por',
           'Descripción del producto', 'Presentación', 'Presentación/Empaque', 'Pack', 'Porción Sugerida',
           'Azúcares Por Porción', 'Carbohidratos Por Porción', 'Descripción Nutricional', 'Grasas Por Porción',
           'Advertencias de consumo', 'itemId', 'ean', 'imageUrl', 'Value', 'IsAvailable', 'AvailableQuantity',
           'Price', 'ListPrice', 'PriceWithoutDiscount', 'PriceValidUntil', 'Created']

df = pd.DataFrame(columns=columns)
df

,productId,productName,brand,brandId,brandImageUrl,categoryId,productTitle,metaTagDescription,categories,categoriesIds,...,ean,imageUrl,Value,IsAvailable,AvailableQuantity,Price,ListPrice,PriceWithoutDiscount,PriceValidUntil,Created


In [12]:
for product in products:
  print(product['items'][0]['sellers'][0]['commertialOffer']['Installments'][0]['Value'])
  print(product['items'][0]['sellers'][0]['commertialOffer']['PriceValidUntil'])
  print(product['items'][0]['sellers'][0]['commertialOffer']['ListPrice'])
  print(product['items'][0]['sellers'][0]['commertialOffer']['PriceWithoutDiscount'])
  break

279.0
2100-01-01T04:59:59Z
279.0
279.0


In [13]:
print(json.dumps(products[3099], indent=4))

{
    "productId": "1050203",
    "productName": "Pack x3 Sixpack Galletas Field Sabor Vainilla 37g",
    "brand": "Field",
    "brandId": 324,
    "brandImageUrl": null,
    "linkText": "pack-x3-sixpack-galletas-field-sabor-vainilla-37g-1016510",
    "productReference": "1016510",
    "productReferenceCode": "1016510",
    "categoryId": "1001429",
    "productTitle": "Pack x3 Sixpack Galletas Field Sabor Vainilla 37g",
    "metaTagDescription": "\u00a1Encuentra aqu\u00ed los mejores productos de Galletas de la marca FIELD!",
    "releaseDate": "2024-08-31T00:00:00Z",
    "clusterHighlights": {
        "6781": "Agenda Del Ahorro Metro Galletas",
        "7035": "TC Abarrotes y Perecibles",
        "7045": "Galletas Metro Abril",
        "7083": "Agenda Ahorro Wong Lunes",
        "7086": "Agenda Ahorro Metro Lunes",
        "7417": "Metro Galletas y Snacks TC",
        "10741": "Dinamica Paneton Metro",
        "10899": "Delivery Gratis T121 Playas",
        "10901": "Delivery Gratis T

In [14]:
# print(json.dumps(products, indent=4))

In [15]:
def print_value(value, key):
    if value:
        print(f"{key} = {value}")
    else:
        print(f"{key} = None")

In [16]:
from bs4 import BeautifulSoup

def clean_html(html_code):
    if html_code is None:
        return ''
    if isinstance(html_code, list):
        html_code = ' '.join(html_code)
    soup = BeautifulSoup(html_code, 'html.parser')
    return soup.get_text()

In [17]:
def get_product_data(product, fecha_db):
    product['Created'] = fecha_db
    new_product = {}
    new_product['productId'] = product.get('productId')
    new_product['productName'] = product.get('productName')
    new_product['brand'] = product.get('brand')
    new_product['brandId'] = product.get('brandId')
    new_product['brandImageUrl'] = product.get('brandImageUrl')
    new_product['categoryId'] = product.get('categoryId')
    new_product['productTitle'] = product.get('productTitle')
    new_product['metaTagDescription'] = product.get('metaTagDescription')
    new_product['categories'] = product.get('categories')
    new_product['categoriesIds'] = product.get('categoriesIds')
    new_product['link'] = product.get('link')
    new_product['Vendido por'] = product.get('Vendido por')
    new_product['Descripción del producto'] = clean_html(product.get('Descripción del producto'))
    new_product['Presentación'] = product.get('Presentación')
    new_product['Presentación/Empaque'] = product.get('Presentación/Empaque')
    new_product['Pack'] = product.get('Pack')
    new_product['Porción Sugerida'] = product.get('Porción Sugerida')
    new_product['Azúcares Por Porción'] = product.get('Azúcares Por Porción')
    new_product['Carbohidratos Por Porción'] = product.get('Carbohidratos Por Porción')
    new_product['Descripción Nutricional'] = product.get('Descripción Nutricional')
    new_product['Grasas Por Porción'] = product.get('Grasas Por Porción')
    new_product['Advertencias de consumo'] = product.get('Advertencias de consumo')
    
    if product['items']:
        new_product['itemId'] = product['items'][0].get('itemId', None)
        new_product['ean'] = product['items'][0].get('ean', None)
        if product['items'][0]['images']:
            new_product['imageUrl'] = product['items'][0]['images'][0].get('imageUrl', None)
            if product['items'][0]['sellers']:
                commertial_offer = product['items'][0]['sellers'][0].get('commertialOffer', {})
                if commertial_offer.get('Installments'):
                    new_product['Value'] = commertial_offer['Installments'][0].get('Value', None)
                new_product['IsAvailable'] = commertial_offer.get('IsAvailable', None)
                new_product['AvailableQuantity'] = commertial_offer.get('AvailableQuantity', None)
                new_product['Price'] = commertial_offer.get('Price', None)
                new_product['ListPrice'] = commertial_offer.get('ListPrice', None)
                new_product['PriceWithoutDiscount'] = commertial_offer.get('PriceWithoutDiscount', None)
                new_product['PriceValidUntil'] = commertial_offer.get('PriceValidUntil', None)

    new_product['Created'] = product['Created']
    
    return new_product

In [18]:
# Para validar que los datos se están extrayendo correctamente

i = 0
for product in products:
    i += 1
    product['Created'] = fecha_hora_lima_db()
    print_value(product.get('productId'), 'productId')
    print_value(product.get('productName'), 'productName')
    print_value(product.get('brand'), 'brand')
    print_value(product.get('brandId'), 'brandId')
    print_value(product.get('brandImageUrl'), 'brandImageUrl')
    print_value(product.get('categoryId'), 'categoryId')
    print_value(product.get('productTitle'), 'productTitle')
    print_value(product.get('metaTagDescription'), 'metaTagDescription')
    print_value(product.get('categories'), 'categories')
    print_value(product.get('categoriesIds'), 'categoriesIds')
    print_value(product.get('link'), 'link')
    print_value(product.get('Vendido por'), 'Vendido por')
    print_value(clean_html(product.get('Descripción del producto')), 'Descripción del producto')
    print_value(product.get('Presentación'), 'Presentación')
    print_value(product.get('Presentación/Empaque'), 'Presentación/Empaque')
    print_value(product.get('Pack'), 'Pack')
    print_value(product.get('Porción Sugerida'), 'Porción Sugerida')
    print_value(product.get('Azúcares Por Porción'), 'Azúcares Por Porción')
    print_value(product.get('Carbohidratos Por Porción'), 'Carbohidratos Por Porción')
    print_value(product.get('Descripción Nutricional'), 'Descripción Nutricional')
    print_value(product.get('Grasas Por Porción'), 'Grasas Por Porción')
    print_value(product.get('Advertencias de consumo'), 'Advertencias de consumo')
    print_value(product['items'][0]['itemId'], 'itemId')
    print_value(product['items'][0]['ean'], 'ean')
    print_value(product['items'][0]['images'][0]['imageUrl'], 'imageUrl')
    print_value(product['items'][0]['sellers'][0]['commertialOffer']['Installments'][0]['Value'], 'Value')
    print_value(product['items'][0]['sellers'][0]['commertialOffer']['IsAvailable'], 'IsAvailable')
    print_value(product['items'][0]['sellers'][0]['commertialOffer']['AvailableQuantity'], 'AvailableQuantity')
    print_value(product['items'][0]['sellers'][0]['commertialOffer']['Price'], 'Price')
    print_value(product['items'][0]['sellers'][0]['commertialOffer']['ListPrice'], 'ListPrice')
    print_value(product['items'][0]['sellers'][0]['commertialOffer']['PriceWithoutDiscount'], 'PriceWithoutDiscount')
    print_value(product['items'][0]['sellers'][0]['commertialOffer']['PriceValidUntil'], 'PriceValidUntil')
    print_value(product['Created'], 'Created')
    
    if i == 3:
        break

productId = 939222
productName = Fruta Deshidratada Manzana, Fresa y Melón Natifrut x kg
brand = Genérico
brandId = 9619
brandImageUrl = None
categoryId = 1001317
productTitle = Fruta Deshidratada Manzana, Fresa y Melón Natifrut x kg
metaTagDescription = ¡Encuentra aquí la mejor variedad de Frutos Secos a Granel de la marca Natifrut!
categories = ['/Frutas y Verduras/Frutas/Frutos Secos/', '/Frutas y Verduras/Frutas/', '/Frutas y Verduras/']
categoriesIds = ['/800/801/1001317/', '/800/801/', '/800/']
link = https://www.wong.pe/fruta-deshidratada-manzana-fresa-y-melon-natifrut-x-kg-771048-2/p
Vendido por = None
Descripción del producto = None
Presentación = None
Presentación/Empaque = None
Pack = None
Porción Sugerida = None
Azúcares Por Porción = None
Carbohidratos Por Porción = None
Descripción Nutricional = None
Grasas Por Porción = None
Advertencias de consumo = None
itemId = 39179921
ean = 22415
imageUrl = https://wongfood.vteximg.com.br/arquivos/ids/509042/Mix-Manzana-Fresa-Mel-n-

In [19]:
print(len(products))

28628


In [20]:
i=0
for product in products:
    i+=1
    if i % 100 == 0:
        print("Iteración #: ",i)
    new_product = get_product_data(product, fecha_hora_para_base_de_datos)
    df.loc[len(df)] = new_product

Iteración #:  100
Iteración #:  200
Iteración #:  300
Iteración #:  400
Iteración #:  500
Iteración #:  600
Iteración #:  700
Iteración #:  800
Iteración #:  900
Iteración #:  1000
Iteración #:  1100
Iteración #:  1200
Iteración #:  1300
Iteración #:  1400
Iteración #:  1500
Iteración #:  1600
Iteración #:  1700
Iteración #:  1800
Iteración #:  1900
Iteración #:  2000
Iteración #:  2100
Iteración #:  2200
Iteración #:  2300
Iteración #:  2400
Iteración #:  2500
Iteración #:  2600
Iteración #:  2700
Iteración #:  2800
Iteración #:  2900
Iteración #:  3000
Iteración #:  3100
Iteración #:  3200
Iteración #:  3300
Iteración #:  3400
Iteración #:  3500
Iteración #:  3600
Iteración #:  3700
Iteración #:  3800
Iteración #:  3900
Iteración #:  4000
Iteración #:  4100
Iteración #:  4200
Iteración #:  4300
Iteración #:  4400
Iteración #:  4500
Iteración #:  4600
Iteración #:  4700
Iteración #:  4800
Iteración #:  4900
Iteración #:  5000
Iteración #:  5100
Iteración #:  5200
Iteración #:  5300
It

In [21]:
df.sample(n=5)

,productId,productName,brand,brandId,brandImageUrl,categoryId,productTitle,metaTagDescription,categories,categoriesIds,...,ean,imageUrl,Value,IsAvailable,AvailableQuantity,Price,ListPrice,PriceWithoutDiscount,PriceValidUntil,Created
11609,956693,Protector Solar para Labios FPS 30 Sun Bum Pom...,Sun Bum,2002560,None,1001212,Protector Solar para Labios FPS 30 Sun Bum Pom...,¡Encuentra aquí la mejor variedad de Protecció...,"[/Higiene, Salud y Belleza/Cuidado Facial y Co...","[/2200/1001208/1001212/, /2200/1001208/, /2200/]",...,871760003620,https://wongfood.vteximg.com.br/arquivos/ids/5...,20.9,True,100,20.9,20.9,20.9,2100-01-01T04:59:59Z,2024-11-13 15:14:45
8893,963883,Vodka Skyy Botella 1L,Skyy,1877,None,2105,Vodka Skyy Botella 1L,¡Encuentra aquí la mejor variedad de Vodka de ...,"[/Cervezas, Vinos y Licores/Licores/Vodka/, /C...","[/2100/1001153/2105/, /2100/1001153/, /2100/]",...,721059010009,https://wongfood.vteximg.com.br/arquivos/ids/6...,52.9,True,100,52.9,71.9,52.9,2100-01-01T04:59:59Z,2024-11-13 15:14:45
12130,1050955,Smartphone Moto edge 50 pro Lavanda + Motobuds,Motorola,2000605,None,1000231,Smartphone Moto edge 50 pro Lavanda + Motobuds,¡Encuentra aquí la mejor variedad en productos...,[/Tecnología/Telefonía/Celulares y Smartphones...,"[/1000144/1000230/1000231/, /1000144/1000230/,...",...,840023267130,https://wongfood.vteximg.com.br/arquivos/ids/7...,2199.0,True,10,2199.0,2699.0,2199.0,2025-11-13T20:15:32Z,2024-11-13 15:14:45
18430,19851,Colchón Paraíso Tropical Ergo 1.5 Plazas,Paraiso,2000467,None,1001088,Colchón Paraíso Tropical Ergo 1.5 Plazas,¡Encuentra aquí la mejor variedad en productos...,"[/Dormitorio/Colchones/Colchones 1.5 Plazas/, ...","[/1001074/1001075/1001088/, /1001074/1001075/,...",...,2050003740944,https://wongfood.vteximg.com.br/arquivos/ids/7...,399.0,True,100,399.0,599.0,399.0,2025-11-13T20:24:33Z,2024-11-13 15:14:45
3308,552239,Salsa Tamarindo Jorvic Botella 350 ml,Jorvic,451,None,1001434,None,None,"[/Abarrotes/Condimentos, Vinagres y Comida Ins...","[/1700/1001294/1001434/, /1700/1001294/, /1700/]",...,7753497140478,https://wongfood.vteximg.com.br/arquivos/ids/3...,12.5,True,100,12.5,12.5,12.5,2100-01-01T04:59:59Z,2024-11-13 15:14:45


In [22]:
df[['productId', 'productName', 'brand', 'categoryId', 'link', 'Price', 'AvailableQuantity']].sample(n=8)

,productId,productName,brand,categoryId,link,Price,AvailableQuantity
24454,967574,Queso Manchego Garcia Baquero x 3kg,García Baquero,1001453,https://www.wong.pe/queso-manchego-garcia-baqu...,245.00,100
15,1050119,Twelvepack Cold Pressed Zuma Perú Shot Defense...,Zuma,1001318,https://www.wong.pe/twelvepack-cold-pressed-zu...,60.00,99999
4130,557256,Caramelos de Hierbas Suizas Limón Menta Sin Az...,Ricola,1001431,https://www.wong.pe/caramelos-de-hierbas-suiza...,7.40,100
26066,1044403,Set Lego Technic Off Road Race Buggy 219 Piezas,Lego,1001168,https://www.wong.pe/set-lego-technic-off-road-...,111.31,10
11527,1014117,Esmalte Sally Hansen Instadri Heather,Sally Hansen,1001231,https://www.wong.pe/esmalte-sally-hansen-insta...,22.20,10
11287,9589,Pond's Crema Humectante y Nutritiva S Pote 100 g,Ponds,1001209,https://www.wong.pe/pond-s-crema-humectante-y-...,25.90,100
3582,14807,Sazonador Adobo Badia 198.4g,Badia,1001304,https://www.wong.pe/sazonador-adobo-badia-198-...,10.30,100
15489,559989,Twopack Pañales para Bebé Pampers Premium Care...,Pampers,2412,https://www.wong.pe/twopack-panales-para-bebe-...,157.90,99999


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28628 entries, 0 to 28627
Data columns (total 33 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   productId                  28628 non-null  object 
 1   productName                28628 non-null  object 
 2   brand                      28628 non-null  object 
 3   brandId                    28628 non-null  int64  
 4   brandImageUrl              0 non-null      object 
 5   categoryId                 28628 non-null  object 
 6   productTitle               23189 non-null  object 
 7   metaTagDescription         19436 non-null  object 
 8   categories                 28628 non-null  object 
 9   categoriesIds              28628 non-null  object 
 10  link                       28628 non-null  object 
 11  Vendido por                0 non-null      object 
 12  Descripción del producto   28628 non-null  object 
 13  Presentación               217 non-null    object 


In [24]:
df['product_uri_json'] = df['productId'].apply(lambda x: f'https://www.wong.pe/api/catalog_system/pub/products/search?fq=productId:{x}')

In [25]:
df['product_uri_json'].sample(n=5)

10480    https://www.wong.pe/api/catalog_system/pub/pro...
13855    https://www.wong.pe/api/catalog_system/pub/pro...
13458    https://www.wong.pe/api/catalog_system/pub/pro...
6395     https://www.wong.pe/api/catalog_system/pub/pro...
5481     https://www.wong.pe/api/catalog_system/pub/pro...
Name: product_uri_json, dtype: object

In [26]:
file_name_productos_csv = f'productos_{fecha_hora_para_archivos}.csv'
df.to_csv(file_name_productos_csv, index=False)

In [27]:
df.iloc[26955]

productId                                                              1034041
productName                        Tapete Plegable Evaonly Unicornios 90x120cm
brand                                                                  Evaonly
brandId                                                                   9882
brandImageUrl                                                             None
categoryId                                                             1001171
productTitle                       Tapete Plegable Evaonly Unicornios 90x120cm
metaTagDescription           ¡Encuentra aquí los mejores productos de Jugue...
categories                   [/La Juguetería/Juguetes Infantiles/Juguetes P...
categoriesIds                [/1001467/1001164/1001171/, /1001467/1001164/,...
link                         https://www.wong.pe/tapete-plegable-evaonly-un...
Vendido por                                                               None
Descripción del producto                            